In [19]:
# 이 코드에서는 깨끗한 단어를 추출하기 위해 심화단계의 tokenize를 수행해 봅니다. (NLP를 위한 전처리)
# 그동안 split()만 사용해 왔지만 다른 방법도 적용해 봅니다. 

from konlpy.tag import Kkma # 형태소 분석기 중에는 kkma가 성능이 제일 좋음.
ma = Kkma() # 형태소 분석기 인스턴스
sentence = "오늘 미세먼지는 어제 미세먼지보다 나빠요."

# 명사를 뽑으라고 시키면 => 오늘, 미세, 먼지, 어제, 미세, 먼지가 뽑혀야 함.
# ma.pos(sentence) # 형태소가 부착된(태깅된) 형태로 반환
print([token[0] for token in ma.pos(sentence) if token[1].startswith("NN")])
print(ma.nouns(sentence)) # 명사만 뽑기

# BOW => index term, Lexicon(dictionary)로 부르기도 함. 
# 문장 단위 -> 어절 단위 -> 형태소 단위 -> 품사(명사) 단위로 보고, 추가로 Ngram 단위로도 문장을 볼 예정
# 일반적인 전처리 순서 : 토크나이징 -> Normalization
from nltk.tokenize import sent_tokenize, word_tokenize # 두 개의 토큰화 모듈 임포트 

# 구두점에 대한 처리만 빼고 비슷한 결과가 나올 것임.
print(sentence.split()) # 단순히 split
print(word_tokenize(sentence)) # 구두점도 별도로 분류를 했으므로 어절이 누구인지 찾을 수 있다.

print("원본 : ", sentence)

# tokenized data
lexicon = list()
lexicon = word_tokenize(sentence)
print("어절 단위 분리 : ", lexicon)

# 품사에 대해 태깅 작업 수행
# print(ma.pos(sentence)) # 둘의 차이는 그렇게 크지 않다.
for token in word_tokenize(sentence): # 명사 분류시 문제가 될 수 있으므로 토큰화한 후 수행
    lexicon.extend([token[0] for token in ma.pos(token)])
print("형태소 분석 결과 : ", list(set(lexicon))) # 필요없는 중복값을 날리기 위해 set()에 담음 
    
lexicon

['오늘', '미세', '먼지', '미세', '먼지']
['오늘', '미세', '미세먼지', '먼지']
['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요.']
['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요', '.']
원본 :  오늘 미세먼지는 어제 미세먼지보다 나빠요.
어절 단위 분리 :  ['오늘', '미세먼지는', '어제', '미세먼지보다', '나빠요', '.']
형태소 분석 결과 :  ['.', '미세먼지는', '아요', '보다', '는', '어제', '먼지', '나쁘', '미세먼지보다', '미세', '오늘', '나빠요']


['오늘',
 '미세먼지는',
 '어제',
 '미세먼지보다',
 '나빠요',
 '.',
 '오늘',
 '미세',
 '먼지',
 '는',
 '어제',
 '미세',
 '먼지',
 '보다',
 '나쁘',
 '아요',
 '.']